In [ ]:
# !pip install torch-geometric
# !pip install gplearn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 2.2 MB/s eta 0:00:00a 0:00:01


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import MessagePassing

In [ ]:
time_data = np.arange(0, 10, 0.1)
g = 9.81

position_data = 1000 - 0.5 * g * time_data **2

# print(position_data)

# plt.plot(time_data, position_data)
# plt.show

In [ ]:
# Define the GNN model
class FreeFallGNN(MessagePassing):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(FreeFallGNN, self).__init__(aggr='mean')
        self.fc1 = nn.Linear(in_channels, hidden_channels)
        self.fc2 = nn.Linear(hidden_channels, hidden_channels)  # Added hidden layer
        self.fc3 = nn.Linear(hidden_channels, out_channels)  # Additional layer
        self.relu = nn.ReLU()

    def forward(self, x, edge_index):
        # Passing the input through the neural network layers
        velocity_update = self.fc1(x)
        velocity_update = self.relu(velocity_update)
        velocity_update = self.fc2(velocity_update)
        velocity_update = self.relu(velocity_update)
        velocity_update = self.fc3(velocity_update)  # Predict velocity update as message
        return velocity_update

# Function to generate simulation data over time (with velocity updates)
def generate_fall_data_from_position(position_data, time_data, mass=1.0, acceleration=-9.81, dt=0.1):
    data_list = []
    velocity_data = np.gradient(position_data, time_data)  # Use np.gradient for velocity estimation
    for i in range(len(position_data) - 1):  # We stop before the last index
        # Calculate velocity update (delta v) as velocity[i+1] - velocity[i]
        velocity_update = velocity_data[i+1] - velocity_data[i]

        x = torch.tensor([[position_data[i], velocity_data[i], mass]], dtype=torch.float)  # Node features (position, velocity, mass)
        y = torch.tensor([[velocity_update]], dtype=torch.float)  # Ground truth velocity update (delta v)
        edge_index = torch.tensor([[0], [0]], dtype=torch.long)  # Self-loop (no interaction)
        data_list.append(Data(x=x, edge_index=edge_index, y=y))  # Only keep velocity update in the label (y)

    return data_list


In [ ]:
# Generate the simulation data from position data
data_list = generate_fall_data_from_position(position_data, time_data)
dataloader = DataLoader(data_list, batch_size=1, shuffle=False)  # Keep sequential order

# Initialize model, loss function, and optimizer
model = FreeFallGNN(in_channels=3, hidden_channels=8, out_channels=1)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.1)

# Training loop
for epoch in range(10):
    total_loss = 0
    for data in dataloader:
        optimizer.zero_grad()
        pred_velocity_update = model(data.x, data.edge_index)  # Predict velocity update
        loss = criterion(pred_velocity_update, data.y)  # Compare with ground truth velocity update
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(dataloader):.6f}")

# Output results
print("\nSimulation Results:")
for idx, data in enumerate(dataloader):
    with torch.no_grad():
        pred_velocity_update = model(data.x, data.edge_index)

        # Calculate the time step: Δt = time[i+1] - time[i]
        delta_t = time_data[idx+1] - time_data[idx]  # Time difference between current and next step

        # Calculate the predicted velocity: v[i+1] = v[i] + Δv
        predicted_velocity = data.x[0][1].item() + pred_velocity_update.item()

        # Calculate the predicted force: F = Δv / Δt
        force = pred_velocity_update.item() / delta_t  # F = predicted Δv / Δt

        print(f"Ground Truth Velocity Update: {data.y.item()}, Predicted Velocity Update: {pred_velocity_update.item()}, Predicted Force: {force:.6f}")